## Global Sensitivity and Uncertainty Analysis

The GSUA approach is based on the work of  A. Carmona-Cabrero and R. Muñoz-Carpena, University of Florida. The paper can be found  in JASS:  [(Carmona-Cabrero et al., 2024)](https://doi.org/10.18564/jasss.5174) 



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from SALib.sample import sobol
from ring_plot_funct import ring_plot
import json
from pathlib import Path

# Folder containing config files
CONFIG_DIR = Path("/blue/carpena/haasehelen/ifwaste/input/gsua_based_configuration")


In [2]:
def extract_parameters(d, path=""):
    result = {}
    for key, value in d.items():
        current_path = f"{path}:{key}" if path else key

        if isinstance(value, dict):
            if "value" in value and "layer" in value:
                result[current_path] = {
                    "value": value.get("value", "NAN"),
                    "distribution":  value.get("distribution", "NAN"),
                    "bounds":  value.get("bounds", "NAN"),
                    "layer": value["layer"],
                    "dtype": value.get("dtype", "NAN"),
                    "length": value.get("length", "NAN"),
                    "options":value.get("options", "NAN"),
                    "decimals": value.get("decimals", "NAN")
                }
            elif "distribution" in value:
                result[current_path] = {
                    "value":  value.get("value", "NAN"),
                    "distribution": value["distribution"],
                    "bounds":  value.get("bounds", "NAN"),
                    "layer": value.get("layer", value["distribution"]),
                    "dtype": value.get("dtype", "NAN"),
                    "length": value.get("length", "NAN"),
                    "options":value.get("options", "NAN"),
                    "decimals": value.get("decimals", "NAN")
                }
            else:
                nested_result = extract_parameters(value, current_path)
                result.update(nested_result)
    return result

## 1. First read all parameterization values from the json files in the CONFIG_DIR
Generate 4 dfs: 
- constants: all parameters that are constants
- parameter: all parameters that shall be sampled using sobol split into:
  - simple_distributions: all parameters that are either triangular or uniform
  - other_distribtuions: parameters, that require extra attention, as they have to be converted to a "simple" distribution


In [3]:
all_json_files = list(CONFIG_DIR.glob("*.json"))
df_list = []
for json_file in all_json_files:
    with open(json_file) as f:
        config = json.load(f)
    flat_params = extract_parameters(config)
    df = pd.DataFrame.from_dict(flat_params, orient="index")
    df.index.name = "name"
    df.reset_index(inplace=True)
    df.replace("", "NAN", inplace=True)
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)

constants = df[df["value"] != "NAN"]
parameter = df[df["value"] == "NAN"]
mask = (parameter["distribution"] == "triang") | (parameter["distribution"] == "unif")
simple_distributions = parameter[mask ]
other_distributions = parameter[~(mask)]



In [4]:
other_distributions.head(100)

,name,value,distribution,bounds,layer,dtype,length,options,decimals
11,Premium_retailer:Sales:high_stock_discount_int...,NAN,sales,NAN,neighborhood,NAN,NAN,[DISCOUNT10],NAN
14,Premium_retailer:Sales:seasonal_discount,NAN,sales,NAN,neighborhood,NAN,NAN,[DISCOUNT10],NAN
19,Premium_retailer:Sales:clearance_interval_1_di...,NAN,sales,NAN,neighborhood,NAN,NAN,[DISCOUNT10],NAN
27,Convenience_store:Sales:high_stock_discount_in...,NAN,sales,NAN,neighborhood,NAN,NAN,"[DISCOUNT20, DISCOUNT30, DISCOUNT40, DISCOUNT5...",NAN
30,Convenience_store:Sales:seasonal_discount,NAN,sales,NAN,neighborhood,NAN,NAN,"[DISCOUNT10, DISCOUNT20, DISCOUNT30, DISCOUNT4...",NAN
35,Convenience_store:Sales:clearance_interval_1_d...,NAN,sales,NAN,neighborhood,NAN,NAN,"[DISCOUNT10, DISCOUNT20, DISCOUNT30]",NAN
40,Neighborhood:nh_store_amounts,NAN,vector,"[0, 1]",neighborhood,int,3,NAN,NAN
94,Household:hh_max_avail_time_per_day,NAN,vector,"[15, 180]",household,int,7,NAN,NAN
97,Household:hh_shopping_frequency,NAN,subgroup,"{'1-2': 0.08, '3-6': 0.12, '7-10': 0.8}",household,int,NAN,NAN,NAN
99,Household:hh_pay_day_interval,NAN,subgroup,"{'14': 0.628, '30': 0.103, '7': 0.269}",household,int,NAN,NAN,NAN


In [5]:
#sim = df[df["layer"] == "simulation"]
#sim = sim.drop(columns=["bounds", "distribution", "layer", "length", "options"])
#sim.to_latex(index=False)

In [6]:
#nh = df[df["layer"] == "neighborhood"]
#nh = nh.drop(columns=["value", "layer", "length", "dtype", "decimals"])
#nh.to_latex(index=False)

In [7]:
#hh = df[df["layer"] == "household"]
#hh = hh.drop(columns=["value", "layer", "length", "options"])
#hh.to_latex()

In [8]:
simple_distributions.head(10)

,name,value,distribution,bounds,layer,dtype,length,options,decimals
9,Premium_retailer:Sales:high_stock_interval_1,NAN,unif,"[1.3, 1.5]",neighborhood,float,NAN,NAN,2
13,Premium_retailer:Sales:seasonal_likelihood,NAN,unif,"[0, 0.05]",neighborhood,float,NAN,NAN,2
25,Convenience_store:Sales:high_stock_interval_1,NAN,unif,"[1, 1.5]",neighborhood,float,NAN,NAN,2
29,Convenience_store:Sales:seasonal_likelihood,NAN,unif,"[0, 0.25]",neighborhood,float,NAN,NAN,2
41,Neighborhood:Grid:travel_time_per_cell,NAN,triang,"[3, 7, 0.5]",neighborhood,float,NAN,NAN,1
45,Neighborhood:BasketCurator:increment_likelihood,NAN,unif,"[0.01, 1]",neighborhood,float,NAN,NAN,2
46,Neighborhood:BasketCurator:max_items_quickshop,NAN,unif,"[2, 8]",neighborhood,int,NAN,NAN,NAN
91,Household:hh_amount_children,NAN,unif,"[0, 6]",household,int,NAN,NAN,NAN
92,Household:hh_amount_adults,NAN,unif,"[1, 2]",household,int,NAN,NAN,NAN
93,Household:hh_level_of_concern,NAN,unif,"[0, 1]",household,float,NAN,NAN,2


## 2. Convert all parameters from "other_distribution" into "simple_distribtuions"

In [9]:
#convert other distributions to sobol-sampleable values
for _,row in other_distributions.iterrows(): 
    if row["distribution"] == "subgroup": 
        new_row = pd.DataFrame({"name":row["name"], "distribution":"unif", "bounds":[[0,1]], "dtype":row["dtype"],
                            "decimals":row["decimals"], "layer": row["layer"], "value":"NAN", "options":"NAN"})
        simple_distributions = pd.concat([simple_distributions, new_row], ignore_index=True)
        
    if row["distribution"] == "vector": 
        for i in range(row["length"]): 
            new_row = pd.DataFrame({"name":row["name"]+"_"+str(i), "distribution":"unif", "bounds":[row["bounds"]], "dtype":row["dtype"],
                            "decimals":2, "layer": row["layer"], "value":"NAN", "options":"NAN"})
            simple_distributions = pd.concat([simple_distributions, new_row], ignore_index=True)
            
    if row["distribution"] == "sales": 
        new_row = pd.DataFrame({"name":row["name"], "distribution":"unif", "bounds":[[0,1]], "dtype":"float",
                            "decimals":2, "layer": row["layer"], "value":"NAN", "options":"NAN"})
        #sales werden dann von vorne nach hinten ausgewählt - nicht noch randomisiert - zack 
        simple_distributions = pd.concat([simple_distributions, new_row], ignore_index=True)        
    

## 3. Read all parameters (now simple and other) into a problem definition for sobol sampling
Then round all values according to the json files (dtype + decimals)

## 4. Prepare for HPC
- add sens_class line for HPC
- check if at least one store would be in the neighborhood, otherwise remove entire sample

## 5. Add replications of samples, write everything to the file
- save the values by level
- also save the parameter names in an extra file for later usage

## 6. Convert other_distributions back to original types for simulation

In [10]:
import csv
from pandas.api.types import is_numeric_dtype

l = 4  # Sampling intensity
N = 10
SAVE_DIR = "/blue/carpena/haasehelen/ifwaste/input/gsua_based_configuration/samples/"


def convert_p_to_value_in_subgroup(p, bounds): 
    cumulative = 0.0
    for key, prob in bounds.items():
        if '-' in key:
            start, end = map(int, key.split('-'))
        else:
            start = end = int(key)
        cumulative += prob
        if p <= cumulative:
            n_values = end - start + 1
            width = prob / n_values
            relative_p = p - (cumulative - prob)
            index_in_group = min(int(relative_p / width), n_values - 1)
            return start + index_in_group


problems = {}

for layer in simple_distributions["layer"].unique():
    print(layer)
    df = simple_distributions[simple_distributions["layer"] == layer]
    names = df["name"].to_list()
    distributions = df["distribution"].to_list()
    bounds = df["bounds"].to_list()
    problem = {'num_vars': len(names),
    'names': names,
    'bounds': bounds,
    'dists':distributions
    }
    param_values = pd.DataFrame(sobol.sample(problem, l, calc_second_order=False), columns=names)  # calc_second_order=True for second-order interactions. 
    for param_name in param_values.columns: 
        if param_name not in other_distributions["name"].to_list():
            row = simple_distributions[simple_distributions["name"] == param_name]
            if row["dtype"].values[0] == "int":
                param_values[param_name] = param_values[param_name].round().astype("int")
            elif row["dtype"].values[0] == "float": 
                param_values[param_name] = param_values[param_name].astype("float").round(int(row["decimals"].values[0]))
            
    
    #filter out unperformable sets (no stores in simulation)
    if layer == "neighborhood":
        print(len(param_values))
        param_values = param_values[~((param_values['Neighborhood:nh_store_amounts_0'] + param_values['Neighborhood:nh_store_amounts_1'] + param_values['Neighborhood:nh_store_amounts_2']) == 0)]
        print(len(param_values))
        
        
    #transform other distributions back
    for _,row in other_distributions.iterrows(): 
        if row["distribution"] == "subgroup" and row["name"] in param_values.columns: 
            bounds = row["bounds"]
            df = pd.DataFrame()
            df["probability"] = param_values[row["name"]]
            df["results"] = df["probability"].apply(lambda p: convert_p_to_value_in_subgroup(p, bounds))
            param_values[row["name"]] = df["results"]
        if row["distribution"] == "vector" and f"{row["name"]}_{0}" in param_values.columns:
            columns_to_drop = []
            col_names = [f"{row["name"]}_{i}" for i in range(row["length"])]
            # Create new column with vectors (lists) for each row
            param_values[row["name"]] = param_values[col_names].values.tolist()
            # Queue old columns for dropping
            columns_to_drop.extend(col_names)
            param_values.drop(columns=columns_to_drop, inplace=True)

        if row["distribution"] == "sales"  and row["name"] in param_values.columns: 
            percentage = param_values[row["name"]]
            df = pd.DataFrame()
            # Calculate how many sales to select
            df["number_of_sales"] = round(percentage * len(row["options"])).astype(int)
            df["number_of_sales"] = df["number_of_sales"].clip(lower=1)
            df["selected_sales"] = df["number_of_sales"].apply(lambda n: row["options"][:n])
            param_values[row["name"]] = df["selected_sales"]
            
    for param_name in param_values.columns: 
        if param_name not in simple_distributions["name"].to_list() and is_numeric_dtype(param_values[param_name]):
            row = simple_distributions[simple_distributions["name"] == param_name]
            if row["dtype"].values[0] == "int":
                param_values[param_name] = param_values[param_name].round().astype("int")
            elif row["dtype"].values[0] == "float": 
                param_values[param_name] = param_values[param_name].astype("float").round(int(row["decimals"].values[0]))
            
    print("GSA sample matrix size: {}".format(param_values.shape))  
    #for HPC added "sens_class" line
    header = param_values.columns.to_list()
    #param_values['sens_class'] = 'sens_class'
    param_values.insert(0, 'sens_class', 'sens_class')
    sample_matrix_realizations = pd.DataFrame(np.repeat(param_values.to_numpy(), repeats=N, axis=0))
    
    print("GSA sample matrix size: {}".format(sample_matrix_realizations.shape))
    #write the sample matrix to csv to configure the model
    sample_matrix_realizations.to_csv( f"{SAVE_DIR}{layer}_ifwaste_sample.txt", header=False, index=False, sep='\t')
    with open(f"{SAVE_DIR}{layer}_ifwaste_sample_header.txt", "w", newline="") as f:
        writer = csv.writer(f)
        for item in header:
            writer.writerow([item])
    
    
constants["value"].to_csv( SAVE_DIR + 'simulation_ifwaste_sample.txt', header=False, index=False, sep='\t')
constants["name"].to_csv( SAVE_DIR + 'simulation_ifwaste_sample_header.txt', index=False, header=False)
        

neighborhood
92
90
GSA sample matrix size: (90, 19)
GSA sample matrix size: (900, 20)
household
GSA sample matrix size: (192, 28)
GSA sample matrix size: (1920, 29)


In [11]:
simple_distributions

,name,value,distribution,bounds,layer,dtype,length,options,decimals
0,Premium_retailer:Sales:high_stock_interval_1,NAN,unif,"[1.3, 1.5]",neighborhood,float,NAN,NAN,2
1,Premium_retailer:Sales:seasonal_likelihood,NAN,unif,"[0, 0.05]",neighborhood,float,NAN,NAN,2
2,Convenience_store:Sales:high_stock_interval_1,NAN,unif,"[1, 1.5]",neighborhood,float,NAN,NAN,2
3,Convenience_store:Sales:seasonal_likelihood,NAN,unif,"[0, 0.25]",neighborhood,float,NAN,NAN,2
4,Neighborhood:Grid:travel_time_per_cell,NAN,triang,"[3, 7, 0.5]",neighborhood,float,NAN,NAN,1
...,...,...,...,...,...,...,...,...,...
62,Child:child_preference_vector_5,NAN,unif,"[0, 1]",household,float,NaN,NAN,2
63,Child:child_preference_vector_6,NAN,unif,"[0, 1]",household,float,NaN,NAN,2
64,Discount_retailer:Sales:high_stock_discount_in...,NAN,unif,"[0, 1]",neighborhood,float,NaN,NAN,2
65,Discount_retailer:Sales:seasonal_discount,NAN,unif,"[0, 1]",neighborhood,float,NaN,NAN,2


In [12]:
param_values    

,sens_class,Household:hh_amount_children,Household:hh_amount_adults,Household:hh_level_of_concern,Household:hh_impulse_buy_likelihood,Household:hh_daily_budget,Household:hh_min_time_to_cook,Household:hh_time_per_store,Household:hh_price_sensitivity,Household:hh_brand_sensitivity,...,Child:child_plate_waste,Child:male_store_prepared_ratio,Child:female_store_prepared_ratio,Cooking:cook_max_scaler_cooking_amount,Cooking:cook_expiration_threshold,Household:hh_shopping_frequency,Household:hh_pay_day_interval,Household:hh_max_avail_time_per_day,Adult:adult_preference_vector,Child:child_preference_vector
0,sens_class,1,2,0.36,1.58,12.96,18,12,0.77,0.16,...,0.275,0.18,0.27,2.29,2.60,1,7,"[107, 64, 130, 78, 69, 65, 82]","[0.31, 0.5, 0.53, 0.66, 0.69, 0.9, 0.41]","[0.56, 0.82, 0.28, 0.12, 0.05, 0.63, 0.47]"
1,sens_class,3,2,0.36,1.58,12.96,18,12,0.77,0.16,...,0.275,0.18,0.27,2.29,2.60,1,7,"[107, 64, 130, 78, 69, 65, 82]","[0.31, 0.5, 0.53, 0.66, 0.69, 0.9, 0.41]","[0.56, 0.82, 0.28, 0.12, 0.05, 0.63, 0.47]"
2,sens_class,1,1,0.36,1.58,12.96,18,12,0.77,0.16,...,0.275,0.18,0.27,2.29,2.60,1,7,"[107, 64, 130, 78, 69, 65, 82]","[0.31, 0.5, 0.53, 0.66, 0.69, 0.9, 0.41]","[0.56, 0.82, 0.28, 0.12, 0.05, 0.63, 0.47]"
3,sens_class,1,2,0.20,1.58,12.96,18,12,0.77,0.16,...,0.275,0.18,0.27,2.29,2.60,1,7,"[107, 64, 130, 78, 69, 65, 82]","[0.31, 0.5, 0.53, 0.66, 0.69, 0.9, 0.41]","[0.56, 0.82, 0.28, 0.12, 0.05, 0.63, 0.47]"
4,sens_class,1,2,0.36,0.65,12.96,18,12,0.77,0.16,...,0.275,0.18,0.27,2.29,2.60,1,7,"[107, 64, 130, 78, 69, 65, 82]","[0.31, 0.5, 0.53, 0.66, 0.69, 0.9, 0.41]","[0.56, 0.82, 0.28, 0.12, 0.05, 0.63, 0.47]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,sens_class,3,1,0.78,0.74,30.60,18,30,0.75,0.37,...,0.160,0.34,0.16,2.69,5.93,10,14,"[92, 113, 160, 151, 34, 176, 33]","[0.59, 0.86, 0.95, 0.86, 0.07, 0.4, 0.54]","[0.47, 0.25, 0.1, 0.03, 0.56, 0.96, 0.99]"
188,sens_class,3,1,0.78,0.74,30.60,18,30,0.75,0.37,...,0.160,0.34,0.16,2.69,5.93,10,14,"[92, 113, 160, 151, 34, 176, 33]","[0.59, 0.86, 0.95, 0.86, 0.07, 0.4, 0.54]","[0.47, 0.25, 0.1, 0.86, 0.81, 0.96, 0.99]"
189,sens_class,3,1,0.78,0.74,30.60,18,30,0.75,0.37,...,0.160,0.34,0.16,2.69,5.93,10,14,"[92, 113, 160, 151, 34, 176, 33]","[0.59, 0.86, 0.95, 0.86, 0.07, 0.4, 0.54]","[0.47, 0.25, 0.1, 0.86, 0.56, 0.15, 0.99]"
190,sens_class,3,1,0.78,0.74,30.60,18,30,0.75,0.37,...,0.160,0.34,0.16,2.69,5.93,10,14,"[92, 113, 160, 151, 34, 176, 33]","[0.59, 0.86, 0.95, 0.86, 0.07, 0.4, 0.54]","[0.47, 0.25, 0.1, 0.86, 0.56, 0.96, 0.92]"


In [13]:
other_distributions.head(100)

,name,value,distribution,bounds,layer,dtype,length,options,decimals
11,Premium_retailer:Sales:high_stock_discount_int...,NAN,sales,NAN,neighborhood,NAN,NAN,[DISCOUNT10],NAN
14,Premium_retailer:Sales:seasonal_discount,NAN,sales,NAN,neighborhood,NAN,NAN,[DISCOUNT10],NAN
19,Premium_retailer:Sales:clearance_interval_1_di...,NAN,sales,NAN,neighborhood,NAN,NAN,[DISCOUNT10],NAN
27,Convenience_store:Sales:high_stock_discount_in...,NAN,sales,NAN,neighborhood,NAN,NAN,"[DISCOUNT20, DISCOUNT30, DISCOUNT40, DISCOUNT5...",NAN
30,Convenience_store:Sales:seasonal_discount,NAN,sales,NAN,neighborhood,NAN,NAN,"[DISCOUNT10, DISCOUNT20, DISCOUNT30, DISCOUNT4...",NAN
35,Convenience_store:Sales:clearance_interval_1_d...,NAN,sales,NAN,neighborhood,NAN,NAN,"[DISCOUNT10, DISCOUNT20, DISCOUNT30]",NAN
40,Neighborhood:nh_store_amounts,NAN,vector,"[0, 1]",neighborhood,int,3,NAN,NAN
94,Household:hh_max_avail_time_per_day,NAN,vector,"[15, 180]",household,int,7,NAN,NAN
97,Household:hh_shopping_frequency,NAN,subgroup,"{'1-2': 0.08, '3-6': 0.12, '7-10': 0.8}",household,int,NAN,NAN,NAN
99,Household:hh_pay_day_interval,NAN,subgroup,"{'14': 0.628, '30': 0.103, '7': 0.269}",household,int,NAN,NAN,NAN
